In [7]:
import ast
import pandas as pd
df = pd.read_csv('/content/split_embedqa.csv')
df.head()

,Context,Questions,Answers,Embedding
0,Berdasarkan barang bukti berikut: 1 (satu) bun...,Berikut adalah 5 pertanyaan dalam bahasa Indon...,Berikut jawaban atas pertanyaan-pertanyaan ter...,"[0.029840383678674698, -0.40641433000564575, -..."
1,Berdasarkan barang bukti berikut: 1 (satu) bun...,NaN,1. Narkotika jenis Pil Ekstasi yang ditemukan ...,"[-0.0014241763856261969, -0.052279382944107056..."
2,Berdasarkan barang bukti berikut: 1 (satu) bun...,1. Berapa gram netto narkotika jenis Pil Eksta...,NaN,"[-0.5292811989784241, -0.2696720361709595, -0...."
3,Berdasarkan barang bukti berikut: 1 (satu) bun...,2. Apakah Terdakwa Nur Afifah Adila Siregar al...,NaN,"[-0.5292811989784241, -0.2696720361709595, -0...."
4,Berdasarkan barang bukti berikut: 1 (satu) bun...,3. Berapa tahun pidana penjara yang dijatuhkan...,NaN,"[-0.5292811989784241, -0.2696720361709595, -0...."


,Context,Questions,Answers,Embedding
0,Berdasarkan barang bukti berikut: 1 (satu) bun...,Berikut adalah 5 pertanyaan dalam bahasa Indon...,Berikut jawaban atas pertanyaan-pertanyaan ter...,"[0.029840383678674698, -0.40641433000564575, -..."
1,Berdasarkan barang bukti berikut: 1 (satu) bun...,NaN,1. Narkotika jenis Pil Ekstasi yang ditemukan ...,"[-0.0014241763856261969, -0.052279382944107056..."
2,Berdasarkan barang bukti berikut: 1 (satu) bun...,1. Berapa gram netto narkotika jenis Pil Eksta...,NaN,"[-0.5292811989784241, -0.2696720361709595, -0...."
3,Berdasarkan barang bukti berikut: 1 (satu) bun...,2. Apakah Terdakwa Nur Afifah Adila Siregar al...,NaN,"[-0.5292811989784241, -0.2696720361709595, -0...."
4,Berdasarkan barang bukti berikut: 1 (satu) bun...,3. Berapa tahun pidana penjara yang dijatuhkan...,NaN,"[-0.5292811989784241, -0.2696720361709595, -0...."


In [8]:
!pip install groq

In [9]:
import os
from groq import Groq
from sentence_transformers import SentenceTransformer

In [10]:
os.environ["GROQ_API_KEY"] = "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"

In [11]:
embedding_model = SentenceTransformer('multi-qa-mpnet-base-dot-v1')

def embedding_query(query):
    try:
        embedding = embedding_model.encode(query)
        return embedding
    except Exception as e:
        print(f"An error occurred during embedding generation: {e}")

def get_answer(query):
    try:
        response = Groq.chat.completions.create(
            messages=[
                {"role": "system", "content": "You are a helpful assistant."},
                {"role": "user", "content": query}
            ],
            max_tokens=150
        )
        return response["choices"][0]["message"]["content"]
    except Exception as e:
        print(f"An error occurred during answer generation: {e}")

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


modules.json:   0%|          | 0.00/229 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/212 [00:00<?, ?B/s]

README.md:   0%|          | 0.00/8.71k [00:00<?, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/53.0 [00:00<?, ?B/s]

...

tokenizer_config.json:   0%|          | 0.00/363 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/239 [00:00<?, ?B/s]

1_Pooling/config.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

In [12]:
from scipy.spatial.distance import cosine
import pandas as pd

def rank_distances(query_embedding, embeddings_list):
    distances = [(i, cosine(query_embedding, ast.literal_eval(embedding))) for i, embedding in enumerate(embeddings_list)]
    return sorted(distances, key=lambda x: x[1])

query = "Apa saja barang bukti yang ditemukan dan dijadikan dasar dalam putusan narkotika?"
query_embedding = embedding_query(query)
ranked_results = rank_distances(query_embedding, df["Embedding"])

# Retrieve the top 5 results from ranked_results with respective ranks and answers
top_5_results = [
    {"No": idx + 1, "Rank": rank, "Answer": df["Answers"][i]}
    for idx, (i, rank) in enumerate(ranked_results[:5])
]

# Create a DataFrame for display
top_5_df = pd.DataFrame(top_5_results)
top_5_df

,No,Rank,Answer
0,1,0.230319,2. Berat narkotika yang ditemukan pada saat pe...
1,2,0.253591,3. Jenis narkotika yang ditemukan dalam 1 plas...
2,3,0.277682,1. Berapa gram kristal putih narkotika jenis s...
3,4,0.283781,1. Berapa gram netto narkotika jenis sabu yang...
4,5,0.285960,1. Jenis narkotika yang ditemukan dalam 2 bung...


,No,Rank,Answer
0,1,0.230319,2. Berat narkotika yang ditemukan pada saat pe...
1,2,0.253591,3. Jenis narkotika yang ditemukan dalam 1 plas...
2,3,0.277682,1. Berapa gram kristal putih narkotika jenis s...
3,4,0.283781,1. Berapa gram netto narkotika jenis sabu yang...
4,5,0.285960,1. Jenis narkotika yang ditemukan dalam 2 bung...
